2019 Census Data for Selected Variables - Baltimore City

In [1]:
#pip install us

In [2]:
# pip install censusgeocode
# pip install censusdata

In [3]:
#conda install -c conda-forge cenpy

In [4]:
#conda update -n base -c defaults conda


In [5]:
# From https://cenpy-devs.github.io/cenpy/:
# Cenpy (pronounced sen-pie) is a package that automatically discovers US Census Bureau API endpoints and exposes them to Python in a consistent fashion. 
# It also provides easy-to-use access to certain well-used data products, like the American Community Survey (ACS) and 2010 Decennial Census.
#pip install cenpy

In [6]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [7]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
import censusgeocode as cg
import cenpy
import gmaps
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2017)

# Configure gmaps
#gmaps.configure(api_key=gkey)

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\libpysal\weights\util.py:23: UserWarning: geopandas not available. Some functionality will be disabled.
  warn("geopandas not available. Some functionality will be disabled.")


In [8]:
# Set this to false if you're trying to do this without an internet connection
# and data which would have been fetched from an API query will be read from cached files instead
INTERNET_IS_WORKING = True

if INTERNET_IS_WORKING:
    con = cenpy.remote.APIConnection('ACSDT5Y2017')
    variables = con.variables
else:
    variables = pd.read_csv('data/ACSDT5Y2017_variables.csv',index_col='Unnamed: 0')

# No matter which product you use, a cenpy APIConnection can show you the variables 
# which it can retrieve for you. They come back as a pandas DataFrame.
print(f"ACSDT5Y2017 provides {len(variables)} variables.") # how many are there?
variables.head()

ACSDT5Y2017 provides 25111 variables.


,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required,values
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN,NaN
B24022_060E,Estimate!!Total!!Female!!Service occupations!!...,SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE P...,int,B24022,0,NaN,NaN,"B24022_060EA,B24022_060M,B24022_060MA",NaN,NaN
B19001B_014E,"Estimate!!Total!!$100,000 to $124,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 201...,int,B19001B,0,NaN,NaN,"B19001B_014EA,B19001B_014M,B19001B_014MA",NaN,NaN


In [9]:
# Comments and code in this block are from https://github.com/censusreporter/nicar20-advanced-census-python/blob/master/workshop.ipynb:
# will use (Nicar20) as citation for the above site from here on out
# values for 'group' are ACS table IDs; 
# for this data, when it's N/A, it's for other kinds of API variables.
# so let's leave those out:
short_vars = variables[~(variables['group'] == 'N/A')] 

# So now this is an easy way to get a list of all of the table IDs and their titles
short_vars[['group', 'concept']].drop_duplicates().sort_values('group').head(10) 

,group,concept
B00001_001E,B00001,UNWEIGHTED SAMPLE COUNT OF THE POPULATION
B00002_001E,B00002,UNWEIGHTED SAMPLE HOUSING UNITS
B01001_012E,B01001,SEX BY AGE
B01001A_002E,B01001A,SEX BY AGE (WHITE ALONE)
B01001B_029E,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
B01001C_008E,B01001C,SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ...
B01001D_008E,B01001D,SEX BY AGE (ASIAN ALONE)
B01001E_013E,B01001E,SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ...
B01001F_001E,B01001F,SEX BY AGE (SOME OTHER RACE ALONE)
B01001G_022E,B01001G,SEX BY AGE (TWO OR MORE RACES)


In [10]:
# (From Nicar20)
# Use when you know which group but still need specific API variable codes
# Attributes column shows related variables you can request. The one that ends with M is the margin of error, and since we want to be responsible when we aggregate data, we'll be sure to aggregate the error as well. 
#The other two, which end with A are "annotations." 
short_vars[short_vars['group'] == 'B01001'][['label','attributes']].sort_index() 

,label,attributes
B01001_001E,Estimate!!Total,"B01001_001EA,B01001_001M,B01001_001MA"
B01001_002E,Estimate!!Total!!Male,"B01001_002EA,B01001_002M,B01001_002MA"
B01001_003E,Estimate!!Total!!Male!!Under 5 years,"B01001_003EA,B01001_003M,B01001_003MA"
B01001_004E,Estimate!!Total!!Male!!5 to 9 years,"B01001_004EA,B01001_004M,B01001_004MA"
B01001_005E,Estimate!!Total!!Male!!10 to 14 years,"B01001_005EA,B01001_005M,B01001_005MA"
B01001_006E,Estimate!!Total!!Male!!15 to 17 years,"B01001_006EA,B01001_006M,B01001_006MA"
B01001_007E,Estimate!!Total!!Male!!18 and 19 years,"B01001_007EA,B01001_007M,B01001_007MA"
B01001_008E,Estimate!!Total!!Male!!20 years,"B01001_008EA,B01001_008M,B01001_008MA"
B01001_009E,Estimate!!Total!!Male!!21 years,"B01001_009EA,B01001_009M,B01001_009MA"
B01001_010E,Estimate!!Total!!Male!!22 to 24 years,"B01001_010EA,B01001_010M,B01001_010MA"


In [11]:
# search function
#sample = censusdata.search('acs5', 2019, 'concept', 'age ')
#print(sample)

#  to see list of all tables in the ACS5
# c.acs5.tables()

In [12]:
censusdata.printtable(censusdata.censustable('acs1', 2019, 'B08604'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08604_001E  | WORKER POPULATION FOR WORKPLAC | !! Estimate Total:                                       | int  
-------------------------------------------------------------------------------------------------------------------


In [13]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
# The state FIPS code for MD is 24 and the FIPS code for Balt City is 510; * is to pull data for all census tracts in the 510 FIPs
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_005E",              
                          "B02001_008E",
                          "B03001_003E",
                          "B25008_002E",
                          "B28007_001E",
                          "B25003_002E",                                    
                          "B25003_003E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
census_pd = pd.DataFrame(census_data)
census_pd.head()      

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_005E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract
0,"Census Tract 2710.02, Baltimore city, Maryland",38358.0,4268.0,37.8,19826.0,1508.0,356.0,1468.0,871.0,375.0,...,0.0,62.0,84.0,2496.0,3090.0,951.0,559.0,24,510,271002
1,"Census Tract 2604.02, Baltimore city, Maryland",42231.0,2999.0,34.0,19014.0,600.0,189.0,1397.0,447.0,218.0,...,0.0,130.0,0.0,1117.0,2179.0,300.0,834.0,24,510,260402
2,"Census Tract 2712, Baltimore city, Maryland",135441.0,6661.0,40.8,64262.0,382.0,172.0,3286.0,446.0,1337.0,...,424.0,357.0,282.0,5744.0,4796.0,1949.0,327.0,24,510,271200
3,"Census Tract 2804.04, Baltimore city, Maryland",39479.0,2519.0,32.0,22909.0,485.0,131.0,1245.0,671.0,106.0,...,0.0,142.0,112.0,1127.0,1761.0,389.0,572.0,24,510,280404
4,"Census Tract 901, Baltimore city, Maryland",44904.0,4057.0,41.9,25138.0,959.0,228.0,1863.0,849.0,232.0,...,10.0,91.0,101.0,2348.0,3414.0,967.0,702.0,24,510,090100


In [14]:
# Pull values from the ACS 5 yr. census codes/ variables listed and save them in the variable "age_data"

age_data = c.acs5.state_county_tract(("NAME", "B01001_003E",                                    
                          "B01001_004E",
                          "B01001_005E",
                          "B01001_006E",
                          "B01001_007E",
                          "B01001_008E",
                          "B01001_009E",
                          "B01001_010E",
                          "B01001_011E",
                          "B01001_012E",
                          "B01001_013E",
                          "B01001_014E",
                          "B01001_015E",
                          "B01001_016E",            
                          "B01001_017E",
                          "B01001_018E",
                          "B01001_019E",
                          "B01001_020E",
                          "B01001_021E",
                          "B01001_022E",
                          "B01001_023E",
                          "B01001_024E",
                          "B01001_025E",
                          
                          "B01001_027E",
                          "B01001_028E",
                          "B01001_029E",
                          "B01001_030E",
                          "B01001_031E",
                          "B01001_032E",
                          "B01001_033E",
                          "B01001_034E",
                          "B01001_035E",
                          "B01001_036E",
                          "B01001_037E",
                          "B01001_038E",
                          "B01001_039E",            
                          "B01001_040E",
                          "B01001_041E",
                          "B01001_042E",
                          "B01001_043E",
                          "B01001_044E",
                          "B01001_045E",
                          "B01001_046E",
                          "B01001_047E",
                          "B01001_048E",
                          "B01001_049E"),          
                                     
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
age_pd = pd.DataFrame(age_data)
age_pd.head()      

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract
0,"Census Tract 2710.02, Baltimore city, Maryland",219.0,195.0,206.0,69.0,65.0,12.0,0.0,80.0,75.0,...,147.0,18.0,123.0,92.0,82.0,92.0,81.0,24,510,271002
1,"Census Tract 2604.02, Baltimore city, Maryland",92.0,145.0,82.0,23.0,5.0,22.0,47.0,72.0,60.0,...,85.0,33.0,35.0,11.0,14.0,33.0,0.0,24,510,260402
2,"Census Tract 2712, Baltimore city, Maryland",444.0,389.0,138.0,102.0,0.0,0.0,35.0,17.0,77.0,...,103.0,24.0,123.0,159.0,73.0,54.0,36.0,24,510,271200
3,"Census Tract 2804.04, Baltimore city, Maryland",119.0,101.0,67.0,22.0,25.0,23.0,6.0,28.0,97.0,...,56.0,28.0,37.0,77.0,14.0,29.0,34.0,24,510,280404
4,"Census Tract 901, Baltimore city, Maryland",71.0,75.0,15.0,104.0,29.0,10.0,10.0,63.0,64.0,...,103.0,64.0,74.0,141.0,19.0,106.0,5.0,24,510,090100


In [15]:
# Sum columns of age groups that are under 18 years old for male and female and add new column "Pop. <18 years"
columns_under18 = ["B01001_003E",                                    
                   "B01001_004E",
                   "B01001_005E",
                   "B01001_006E",
                   "B01001_027E",
                   "B01001_028E",
                   "B01001_029E",
                   "B01001_030E"]
age_pd['Pop. <18 years']= age_pd[columns_under18].sum(axis=1)
age_pd.head()

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years
0,"Census Tract 2710.02, Baltimore city, Maryland",219.0,195.0,206.0,69.0,65.0,12.0,0.0,80.0,75.0,...,18.0,123.0,92.0,82.0,92.0,81.0,24,510,271002,1210.0
1,"Census Tract 2604.02, Baltimore city, Maryland",92.0,145.0,82.0,23.0,5.0,22.0,47.0,72.0,60.0,...,33.0,35.0,11.0,14.0,33.0,0.0,24,510,260402,814.0
2,"Census Tract 2712, Baltimore city, Maryland",444.0,389.0,138.0,102.0,0.0,0.0,35.0,17.0,77.0,...,24.0,123.0,159.0,73.0,54.0,36.0,24,510,271200,1709.0
3,"Census Tract 2804.04, Baltimore city, Maryland",119.0,101.0,67.0,22.0,25.0,23.0,6.0,28.0,97.0,...,28.0,37.0,77.0,14.0,29.0,34.0,24,510,280404,590.0
4,"Census Tract 901, Baltimore city, Maryland",71.0,75.0,15.0,104.0,29.0,10.0,10.0,63.0,64.0,...,64.0,74.0,141.0,19.0,106.0,5.0,24,510,090100,793.0


In [16]:
# Sum columns of age groups that are 18-64 years old (working age) for male and female and add new column "Pop. working age"
columns_working_age = [                                    
                   "B01001_007E",
                   "B01001_008E",
                   "B01001_009E",
                   "B01001_010E",
                   "B01001_011E",
                   "B01001_012E",   
                   "B01001_013E",
                   "B01001_014E",    
                   "B01001_015E",    
                   "B01001_016E",   
                   "B01001_017E", 
                   "B01001_018E",    
                   "B01001_019E",    
                   "B01001_031E",
                   "B01001_032E",
                   "B01001_033E",
                   "B01001_034E",
                   "B01001_035E",   
                   "B01001_036E",
                   "B01001_037E",    
                   "B01001_038E",    
                   "B01001_039E",   
                   "B01001_040E", 
                   "B01001_041E",    
                   "B01001_042E",    
                   "B01001_043E"]    
                           
age_pd['Pop. working age']= age_pd[columns_working_age].sum(axis=1)
age_pd.head()         
            
               


,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age
0,"Census Tract 2710.02, Baltimore city, Maryland",219.0,195.0,206.0,69.0,65.0,12.0,0.0,80.0,75.0,...,123.0,92.0,82.0,92.0,81.0,24,510,271002,1210.0,2305.0
1,"Census Tract 2604.02, Baltimore city, Maryland",92.0,145.0,82.0,23.0,5.0,22.0,47.0,72.0,60.0,...,35.0,11.0,14.0,33.0,0.0,24,510,260402,814.0,1951.0
2,"Census Tract 2712, Baltimore city, Maryland",444.0,389.0,138.0,102.0,0.0,0.0,35.0,17.0,77.0,...,123.0,159.0,73.0,54.0,36.0,24,510,271200,1709.0,3964.0
3,"Census Tract 2804.04, Baltimore city, Maryland",119.0,101.0,67.0,22.0,25.0,23.0,6.0,28.0,97.0,...,37.0,77.0,14.0,29.0,34.0,24,510,280404,590.0,1557.0
4,"Census Tract 901, Baltimore city, Maryland",71.0,75.0,15.0,104.0,29.0,10.0,10.0,63.0,64.0,...,74.0,141.0,19.0,106.0,5.0,24,510,090100,793.0,2609.0


In [17]:
# Sum columns of age groups that are 65+ years old for male and female and add new column "Pop. 65+ years"
columns_senior = ["B01001_020E",
                  "B01001_021E",
                  "B01001_022E",
                  "B01001_023E",                                    
                  "B01001_024E",
                  "B01001_025E",
                  "B01001_044E",
                  "B01001_045E",
                  "B01001_046E",
                  "B01001_047E",                                    
                  "B01001_048E",
                  "B01001_049E"]               
                          
age_pd['Pop. 65+ years']= age_pd[columns_senior].sum(axis=1)
age_pd.head()                           

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,"Census Tract 2710.02, Baltimore city, Maryland",219.0,195.0,206.0,69.0,65.0,12.0,0.0,80.0,75.0,...,92.0,82.0,92.0,81.0,24,510,271002,1210.0,2305.0,753.0
1,"Census Tract 2604.02, Baltimore city, Maryland",92.0,145.0,82.0,23.0,5.0,22.0,47.0,72.0,60.0,...,11.0,14.0,33.0,0.0,24,510,260402,814.0,1951.0,234.0
2,"Census Tract 2712, Baltimore city, Maryland",444.0,389.0,138.0,102.0,0.0,0.0,35.0,17.0,77.0,...,159.0,73.0,54.0,36.0,24,510,271200,1709.0,3964.0,988.0
3,"Census Tract 2804.04, Baltimore city, Maryland",119.0,101.0,67.0,22.0,25.0,23.0,6.0,28.0,97.0,...,77.0,14.0,29.0,34.0,24,510,280404,590.0,1557.0,372.0
4,"Census Tract 901, Baltimore city, Maryland",71.0,75.0,15.0,104.0,29.0,10.0,10.0,63.0,64.0,...,141.0,19.0,106.0,5.0,24,510,090100,793.0,2609.0,655.0


In [18]:
age_final = age_pd[[ "tract", "Pop. <18 years", "Pop. working age", "Pop. 65+ years"]]
age_final.head()

,tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,271002,1210.0,2305.0,753.0
1,260402,814.0,1951.0,234.0
2,271200,1709.0,3964.0,988.0
3,280404,590.0,1557.0,372.0
4,090100,793.0,2609.0,655.0


In [19]:
age_final = age_final.rename(columns={"tract": "Census_tract"})
age_final

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years
0,271002,1210.0,2305.0,753.0
1,260402,814.0,1951.0,234.0
2,271200,1709.0,3964.0,988.0
3,280404,590.0,1557.0,372.0
4,090100,793.0,2609.0,655.0
...,...,...,...,...
195,230300,33.0,1187.0,86.0
196,250207,521.0,1351.0,221.0
197,250303,490.0,1439.0,295.0
198,260202,1365.0,4019.0,401.0


In [20]:
# did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state,county
0,"Baltimore city, Maryland",379990.0,24,510


In [21]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract #
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract,GEOID
0,"Census Tract 2710.02, Baltimore city, Maryland",38358.0,4268.0,37.8,19826.0,1508.0,356.0,1468.0,871.0,375.0,...,62.0,84.0,2496.0,3090.0,951.0,559.0,24,510,271002,24510271002
1,"Census Tract 2604.02, Baltimore city, Maryland",42231.0,2999.0,34.0,19014.0,600.0,189.0,1397.0,447.0,218.0,...,130.0,0.0,1117.0,2179.0,300.0,834.0,24,510,260402,24510260402
2,"Census Tract 2712, Baltimore city, Maryland",135441.0,6661.0,40.8,64262.0,382.0,172.0,3286.0,446.0,1337.0,...,357.0,282.0,5744.0,4796.0,1949.0,327.0,24,510,271200,24510271200
3,"Census Tract 2804.04, Baltimore city, Maryland",39479.0,2519.0,32.0,22909.0,485.0,131.0,1245.0,671.0,106.0,...,142.0,112.0,1127.0,1761.0,389.0,572.0,24,510,280404,24510280404
4,"Census Tract 901, Baltimore city, Maryland",44904.0,4057.0,41.9,25138.0,959.0,228.0,1863.0,849.0,232.0,...,91.0,101.0,2348.0,3414.0,967.0,702.0,24,510,090100,24510090100


In [22]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 23)


In [23]:
# remove extraneous column "tract"
census_pd = census_pd.drop(["tract"], axis=1)
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_005E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,GEOID
0,"Census Tract 2710.02, Baltimore city, Maryland",38358.0,4268.0,37.8,19826.0,1508.0,356.0,1468.0,871.0,375.0,...,0.0,62.0,84.0,2496.0,3090.0,951.0,559.0,24,510,24510271002
1,"Census Tract 2604.02, Baltimore city, Maryland",42231.0,2999.0,34.0,19014.0,600.0,189.0,1397.0,447.0,218.0,...,0.0,130.0,0.0,1117.0,2179.0,300.0,834.0,24,510,24510260402
2,"Census Tract 2712, Baltimore city, Maryland",135441.0,6661.0,40.8,64262.0,382.0,172.0,3286.0,446.0,1337.0,...,424.0,357.0,282.0,5744.0,4796.0,1949.0,327.0,24,510,24510271200
3,"Census Tract 2804.04, Baltimore city, Maryland",39479.0,2519.0,32.0,22909.0,485.0,131.0,1245.0,671.0,106.0,...,0.0,142.0,112.0,1127.0,1761.0,389.0,572.0,24,510,24510280404
4,"Census Tract 901, Baltimore city, Maryland",44904.0,4057.0,41.9,25138.0,959.0,228.0,1863.0,849.0,232.0,...,10.0,91.0,101.0,2348.0,3414.0,967.0,702.0,24,510,24510090100


In [24]:
# combine age data with rest of census data
census_joined = pd.concat([age_final, census_pd], axis="columns")
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,...,B02001_005E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,GEOID
0,271002,1210.0,2305.0,753.0,"Census Tract 2710.02, Baltimore city, Maryland",38358.0,4268.0,37.8,19826.0,1508.0,...,0.0,62.0,84.0,2496.0,3090.0,951.0,559.0,24,510,24510271002
1,260402,814.0,1951.0,234.0,"Census Tract 2604.02, Baltimore city, Maryland",42231.0,2999.0,34.0,19014.0,600.0,...,0.0,130.0,0.0,1117.0,2179.0,300.0,834.0,24,510,24510260402
2,271200,1709.0,3964.0,988.0,"Census Tract 2712, Baltimore city, Maryland",135441.0,6661.0,40.8,64262.0,382.0,...,424.0,357.0,282.0,5744.0,4796.0,1949.0,327.0,24,510,24510271200
3,280404,590.0,1557.0,372.0,"Census Tract 2804.04, Baltimore city, Maryland",39479.0,2519.0,32.0,22909.0,485.0,...,0.0,142.0,112.0,1127.0,1761.0,389.0,572.0,24,510,24510280404
4,090100,793.0,2609.0,655.0,"Census Tract 901, Baltimore city, Maryland",44904.0,4057.0,41.9,25138.0,959.0,...,10.0,91.0,101.0,2348.0,3414.0,967.0,702.0,24,510,24510090100


In [25]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's understandable
census_joined = census_joined.rename(columns={"B01003_001E": "Population",
                                      "tract": "Census Tract",        
                                      "B01002_001E": "Median age",
                                      "B19013_001E": "Median household income",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_005E": "Pop. Asian",        
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "B28007_001E": "Labor force status by presence of computer/ internet in HH",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"})

# Add a new column for poverty rate (Poverty Count / Population)
census_joined["Poverty rate"] = 100 * \
    census_joined["Poverty count"].astype(
        int) / census_joined["Population"].astype(int)

# Add a new column for unemployment rate (Employment Count / Population)
census_joined["Unemployment rate"] = 100 * \
    census_joined["Unemployment count"].astype(
        int) / census_joined["Population"].astype(int)
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Labor force status by presence of computer/ internet in HH,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate
0,271002,1210.0,2305.0,753.0,"Census Tract 2710.02, Baltimore city, Maryland",38358.0,4268.0,37.8,19826.0,1508.0,...,84.0,2496.0,3090.0,951.0,559.0,24,510,24510271002,35.332709,8.341143
1,260402,814.0,1951.0,234.0,"Census Tract 2604.02, Baltimore city, Maryland",42231.0,2999.0,34.0,19014.0,600.0,...,0.0,1117.0,2179.0,300.0,834.0,24,510,24510260402,20.006669,6.302101
2,271200,1709.0,3964.0,988.0,"Census Tract 2712, Baltimore city, Maryland",135441.0,6661.0,40.8,64262.0,382.0,...,282.0,5744.0,4796.0,1949.0,327.0,24,510,24510271200,5.734875,2.582195
3,280404,590.0,1557.0,372.0,"Census Tract 2804.04, Baltimore city, Maryland",39479.0,2519.0,32.0,22909.0,485.0,...,112.0,1127.0,1761.0,389.0,572.0,24,510,24510280404,19.253672,5.200476
4,090100,793.0,2609.0,655.0,"Census Tract 901, Baltimore city, Maryland",44904.0,4057.0,41.9,25138.0,959.0,...,101.0,2348.0,3414.0,967.0,702.0,24,510,24510090100,23.638156,5.619916


In [26]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_joined.shape)

Number of rows, columns:  (200, 28)


In [27]:
# Add in home ownership rate (# owner-occupied units / # of occupied housing units)
# sum 2 columns: total owner-occupied units + total renter-occupied units to create additional column "Total occupied units" 
sum_column = census_joined['Total owner-occupied units'] + census_joined['Total renter-occupied units']
census_joined["Total occupied units"] = sum_column

In [28]:
census_joined["Home ownership rate"] = 100 * \
    census_joined["Total owner-occupied units"].astype(
        int) / census_joined["Total occupied units"].astype(
        int) 

census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Labor force status by presence of computer/ internet in HH,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,271002,1210.0,2305.0,753.0,"Census Tract 2710.02, Baltimore city, Maryland",38358.0,4268.0,37.8,19826.0,1508.0,...,3090.0,951.0,559.0,24,510,24510271002,35.332709,8.341143,1510.0,62.980132
1,260402,814.0,1951.0,234.0,"Census Tract 2604.02, Baltimore city, Maryland",42231.0,2999.0,34.0,19014.0,600.0,...,2179.0,300.0,834.0,24,510,24510260402,20.006669,6.302101,1134.0,26.455026
2,271200,1709.0,3964.0,988.0,"Census Tract 2712, Baltimore city, Maryland",135441.0,6661.0,40.8,64262.0,382.0,...,4796.0,1949.0,327.0,24,510,24510271200,5.734875,2.582195,2276.0,85.632689
3,280404,590.0,1557.0,372.0,"Census Tract 2804.04, Baltimore city, Maryland",39479.0,2519.0,32.0,22909.0,485.0,...,1761.0,389.0,572.0,24,510,24510280404,19.253672,5.200476,961.0,40.478668
4,090100,793.0,2609.0,655.0,"Census Tract 901, Baltimore city, Maryland",44904.0,4057.0,41.9,25138.0,959.0,...,3414.0,967.0,702.0,24,510,24510090100,23.638156,5.619916,1669.0,57.938886


In [29]:
# round the home ownership rate to one decimal point; using "float" instead of "int" because want to use decimal points
census_joined["Home ownership rate"] = census_joined["Home ownership rate"].astype(float).round(1)


In [30]:
census_joined["Poverty rate"] = census_joined["Poverty rate"].astype(float).round(1)


In [31]:
census_joined["Unemployment rate"] = census_joined["Unemployment rate"].astype(float).round(1)
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Labor force status by presence of computer/ internet in HH,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,271002,1210.0,2305.0,753.0,"Census Tract 2710.02, Baltimore city, Maryland",38358.0,4268.0,37.8,19826.0,1508.0,...,3090.0,951.0,559.0,24,510,24510271002,35.3,8.3,1510.0,63.0
1,260402,814.0,1951.0,234.0,"Census Tract 2604.02, Baltimore city, Maryland",42231.0,2999.0,34.0,19014.0,600.0,...,2179.0,300.0,834.0,24,510,24510260402,20.0,6.3,1134.0,26.5
2,271200,1709.0,3964.0,988.0,"Census Tract 2712, Baltimore city, Maryland",135441.0,6661.0,40.8,64262.0,382.0,...,4796.0,1949.0,327.0,24,510,24510271200,5.7,2.6,2276.0,85.6
3,280404,590.0,1557.0,372.0,"Census Tract 2804.04, Baltimore city, Maryland",39479.0,2519.0,32.0,22909.0,485.0,...,1761.0,389.0,572.0,24,510,24510280404,19.3,5.2,961.0,40.5
4,090100,793.0,2609.0,655.0,"Census Tract 901, Baltimore city, Maryland",44904.0,4057.0,41.9,25138.0,959.0,...,3414.0,967.0,702.0,24,510,24510090100,23.6,5.6,1669.0,57.9


In [32]:
# Calculate population density  see: https://www.census.gov/quickfacts/fact/note/US/LND110210
# density is expressed as "population per square mile(kilometer)"
# Divide total population (or # of housing units)/ by land area of the entity measured in square miles

In [33]:
census_joined.count()

Census_tract                                                  200
Pop. <18 years                                                200
Pop. working age                                              200
Pop. 65+ years                                                200
Name                                                          200
Median household income                                       200
Population                                                    200
Median age                                                    200
Per capita income                                             200
Poverty count                                                 200
Unemployment count                                            200
# employed, age 16+                                           200
# persons age 25+ graduated high school                       200
# persons age 25+ with Bachelor's degree                      200
Pop. white                                                    200
Pop. Black

In [34]:
# Remove "State" column because it's understood we are looking at MD data
census_joined = census_joined.drop(["Name"], axis=1)
census_joined.head()

,Census_tract,Pop. <18 years,Pop. working age,Pop. 65+ years,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,...,Labor force status by presence of computer/ internet in HH,Total owner-occupied units,Total renter-occupied units,State,county,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,271002,1210.0,2305.0,753.0,38358.0,4268.0,37.8,19826.0,1508.0,356.0,...,3090.0,951.0,559.0,24,510,24510271002,35.3,8.3,1510.0,63.0
1,260402,814.0,1951.0,234.0,42231.0,2999.0,34.0,19014.0,600.0,189.0,...,2179.0,300.0,834.0,24,510,24510260402,20.0,6.3,1134.0,26.5
2,271200,1709.0,3964.0,988.0,135441.0,6661.0,40.8,64262.0,382.0,172.0,...,4796.0,1949.0,327.0,24,510,24510271200,5.7,2.6,2276.0,85.6
3,280404,590.0,1557.0,372.0,39479.0,2519.0,32.0,22909.0,485.0,131.0,...,1761.0,389.0,572.0,24,510,24510280404,19.3,5.2,961.0,40.5
4,090100,793.0,2609.0,655.0,44904.0,4057.0,41.9,25138.0,959.0,228.0,...,3414.0,967.0,702.0,24,510,24510090100,23.6,5.6,1669.0,57.9


In [35]:
# Split the "Name" column into 3 separate columns: "Census_Tract", "County", "State"
#census_joined[['Census_Tract', "County", "State"]]= census_joined['Name'].str.split(",", n=3, expand=True)
#census_joined.head()

In [36]:
# list the columns in the census_pd dataframe
census_joined.columns

Index(['Census_tract', 'Pop. <18 years', 'Pop. working age', 'Pop. 65+ years',
       'Median household income', 'Population', 'Median age',
       'Per capita income', 'Poverty count', 'Unemployment count',
       '# employed, age 16+', '# persons age 25+ graduated high school',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. 2 or more races', 'Pop. Hispanic origin',
       'Total pop. in occupied housing units by tenure',
       'Labor force status by presence of computer/ internet in HH',
       'Total owner-occupied units', 'Total renter-occupied units', 'State',
       'county', 'GEOID', 'Poverty rate', 'Unemployment rate',
       'Total occupied units', 'Home ownership rate'],
      dtype='object')

In [37]:
# Create new column "Census_Tract" and remove the text "Census Tract" from the values in that column (to make calculations easier)
#census_joined["Census_Tract"] = census_joined['Census_Tract'].str.replace('Census Tract', "") 


In [38]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_joined["Census_tract"].unique())
tract_count

200

In [39]:
# Change order of columns in DataFrame by using double brackets
census_joined = census_joined[["Census_tract", "GEOID", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin", 
                      "Pop. Asian","Total pop. in occupied housing units by tenure", "Total owner-occupied units", "Total renter-occupied units",
                      "Pop. <18 years", "Pop. working age", "Pop. 65+ years",         
                      "Labor force status by presence of computer/ internet in HH", "county"]]

census_joined.head()

,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,...,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years,Labor force status by presence of computer/ internet in HH,county
0,271002,24510271002,4268.0,38358.0,19826.0,1508.0,35.3,8.3,1468.0,356.0,...,84.0,0.0,2496.0,951.0,559.0,1210.0,2305.0,753.0,3090.0,510
1,260402,24510260402,2999.0,42231.0,19014.0,600.0,20.0,6.3,1397.0,189.0,...,0.0,0.0,1117.0,300.0,834.0,814.0,1951.0,234.0,2179.0,510
2,271200,24510271200,6661.0,135441.0,64262.0,382.0,5.7,2.6,3286.0,172.0,...,282.0,424.0,5744.0,1949.0,327.0,1709.0,3964.0,988.0,4796.0,510
3,280404,24510280404,2519.0,39479.0,22909.0,485.0,19.3,5.2,1245.0,131.0,...,112.0,0.0,1127.0,389.0,572.0,590.0,1557.0,372.0,1761.0,510
4,090100,24510090100,4057.0,44904.0,25138.0,959.0,23.6,5.6,1863.0,228.0,...,101.0,10.0,2348.0,967.0,702.0,793.0,2609.0,655.0,3414.0,510


In [40]:
# sort the column "Census_tract" so that it is easier to compare with other years that we are going to pull in another notebook
census_2017_FINAL =census_joined.sort_values("Census_tract")
census_2017_FINAL

,Census_tract,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,...,Pop. Hispanic origin,Pop. Asian,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. <18 years,Pop. working age,Pop. 65+ years,Labor force status by presence of computer/ internet in HH,county
99,010100,24510010100,3201.0,95625.0,57582.0,372.0,11.6,1.8,2175.0,59.0,...,58.0,120.0,1701.0,911.0,555.0,199.0,2523.0,479.0,2840.0,510
11,010200,24510010200,3145.0,109102.0,52527.0,185.0,5.9,1.1,2327.0,34.0,...,66.0,138.0,2514.0,1133.0,304.0,367.0,2562.0,216.0,2785.0,510
19,010300,24510010300,2552.0,137275.0,67376.0,103.0,4.0,0.6,1909.0,15.0,...,97.0,104.0,1641.0,733.0,350.0,340.0,2083.0,129.0,2218.0,510
56,010400,24510010400,2573.0,112059.0,83899.0,122.0,4.7,2.8,2019.0,71.0,...,102.0,141.0,1461.0,787.0,617.0,158.0,2146.0,269.0,2403.0,510
100,010500,24510010500,1712.0,92000.0,55437.0,149.0,8.7,0.9,1210.0,15.0,...,105.0,40.0,1076.0,485.0,273.0,209.0,1367.0,136.0,1477.0,510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,280401,24510280401,3606.0,51309.0,28064.0,400.0,11.1,3.8,1694.0,137.0,...,60.0,0.0,2305.0,926.0,573.0,701.0,2300.0,605.0,2982.0,510
86,280402,24510280402,1657.0,40781.0,19845.0,133.0,8.0,5.1,703.0,84.0,...,8.0,0.0,1133.0,452.0,178.0,305.0,970.0,382.0,1386.0,510
87,280403,24510280403,5431.0,60061.0,35590.0,395.0,7.3,4.4,2911.0,240.0,...,167.0,43.0,2585.0,988.0,1236.0,1201.0,3592.0,638.0,4245.0,510
3,280404,24510280404,2519.0,39479.0,22909.0,485.0,19.3,5.2,1245.0,131.0,...,112.0,0.0,1127.0,389.0,572.0,590.0,1557.0,372.0,1761.0,510


In [41]:
# Export file as a CSV, without the Pandas index, but with the header
# Do not run this last code block until you have all previous code blocks in their final form:

#census_2017_FINAL.to_csv("CommCorr_Census_Stats_2017.csv", index = False, header=True)